In [1]:
import pandas as pd
df = pd.read_csv('phase3-var-sel.csv')
df.head()

,MaxLapl,AvgMaxX,MaxVar,LaplVar,SurfNumKP,MaxY,RidgeAreaProp,ORBNumKP,HistMean,MinVar,C1AreaProp,AkazeNumKP,KazeNumKP,MaxAvgLapl,HistVar,Output,Filename
0,255.283676,44643.0,3141.374252,613.496113,103,32988,0.422052,127,172.465515,107.880293,0.353271,4,27,297.757073,720.605256,1,ESP_011314_1585_RED.NOMAP.browse-Block-7.jpg
1,299.942612,595215.0,5497.009506,1075.129234,69,55707,0.238823,27,114.768188,89.108415,0.219666,6,65,601.463473,9042.951879,1,ESP_011325_1845_RED.NOMAP.browse-Block-10.jpg
2,465.874008,378452.0,5698.864609,989.958606,59,36999,0.282823,94,85.229126,141.611445,0.204712,8,61,487.412923,5737.952018,1,ESP_011325_1845_RED.NOMAP.browse-Block-12.jpg
3,374.875717,411315.0,8084.393661,1190.971659,72,42144,0.271832,50,79.706726,182.914684,0.217834,2,60,545.449855,5033.211659,1,ESP_011325_1845_RED.NOMAP.browse-Block-14.jpg
4,109.927353,498016.0,7019.038086,932.756638,65,89109,0.287259,101,93.042786,75.190465,0.218262,5,44,553.311266,6122.234436,1,ESP_011325_1845_RED.NOMAP.browse-Block-16.jpg


In [2]:
df['Output'] = df['Output'].apply(lambda x: 1 if x > 0 else 0)

In [3]:
groups = df['Filename'].apply(lambda x: x.split('.')[0]).astype('category').cat.codes

In [4]:
from sklearn.model_selection import GroupShuffleSplit

main_x, main_y = df.drop(columns=['Filename', 'Output']).values, df['Output'].values

cv = GroupShuffleSplit(n_splits=2, test_size=0.2, random_state=1337)
train_indices, test_indices = next(cv.split(main_x, main_y, groups))
X, Y = main_x[train_indices], main_y[train_indices]
x_test, y_test = main_x[test_indices], main_y[test_indices]

In [5]:
from scipy.stats import uniform
penalty = ['l1', 'l2']
C = uniform(loc=0, scale=4)
hyperparameters = dict(C=C, penalty=penalty)

In [6]:
groups = df.iloc[train_indices]['Filename'].apply(lambda x: x.split('.')[0]).astype('category').cat.codes

In [7]:
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression

cv = GroupKFold(n_splits=10)
clf = RandomizedSearchCV(LogisticRegression(random_state=1337), hyperparameters, random_state=1337, n_iter=100, cv=cv, verbose=0, n_jobs=-1)
hyp = clf.fit(X,Y,groups=groups)

C:\Users\Michael\Anaconda3\envs\machine-learning\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [8]:
hyp.best_estimator_.get_params()

{'C': 0.3857999428894119,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'warn',
 'n_jobs': None,
 'penalty': 'l1',
 'random_state': 1337,
 'solver': 'warn',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [10]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

y_pred = hyp.best_estimator_.predict(x_test)
print("Accuracy: " + str(round(accuracy_score(y_test, y_pred), 2)))
confusion_matrix(y_test, y_pred)

Accuracy: 0.84


array([[184,  19],
       [ 40, 124]], dtype=int64)